In [164]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [165]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [166]:
# API KEYS from environment variables

PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
PLAID_SECRET = os.getenv('PLAID_SECRET')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [167]:
INSTITUTION_ID = "ins_109508"

In [168]:
# Create Client Object 
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment=PLAID_ENV)
# Get institutions
client.Institutions.get(2)

{'institutions': [{'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112060',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Bank (Broadus, MT) - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',
    'balance',
    'transactions',
    'income',
    'identity'],
   'routing_numbers': []},
  {'country_codes': ['US'],
   'credentials': [{'label': 'Username', 'name': 'username', 'type': 'text'},
    {'label': 'Password', 'name': 'password', 'type': 'password'}],
   'has_mfa': True,
   'input_spec': 'fixed',
   'institution_id': 'ins_112062',
   'mfa': ['code', 'list', 'questions', 'selections'],
   'mfa_code_type': 'numeric',
   'name': '1st Constitution Bank - Personal',
   'oauth': False,
   'products': ['assets',
    'auth',


### 2. Generate a public token

In [169]:
# Select Institution for processing 
INSTITUTION_ID = "ins_109508"

# Create Public Token 
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions', 'income', 'assets'])

### 3. Exchange the public token for an access token

In [170]:
# Exchange 
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Store as variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [171]:
# Fetch 
client.Accounts.get(access_token)

{'accounts': [{'account_id': '9zAjEP1EV3TN3nmVomebH55LeAzjqdflPQoME',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'vNPJnxdn5GUEoXQNPQyMCnnx9b5Gdeio5kbVl',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'RK5ADzdDVmTQm6k3AkpjinnNx5ZpbgizdbN5w',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [190]:
# Get transactions for institution for specific date range
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Get transactions for date range
transaction_response = client.Transactions.get(access_token,start_date,end_date)
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "vNPJnxdn5GUEoXQNPQyMCnnx9b5Gdeio5kbVl",
        "account_owner": null,
        "amount": 25,
        "authorized_date": null,
        "category": [
            "Payment",
            "Credit Card"
        ],
        "category_id": "16001000",
        "date": "2020-06-14",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "CREDIT CARD 3333 PAYMENT *//",
        "payment_channel": "other",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": fals

### 2. Print the categories for each transaction

In [191]:
# Print
for transaction in transaction_response['transactions']:
    print(transaction['category'])

['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast Food']
['Shops', 'Sporting Goods']
['Payment', 'Credit Card']
['Travel', 'Taxi']
['Transfer', 'Debit']
['Transfer', 'Deposit']
['Recreation', 'Gyms and Fitness Centers']
['Travel', 'Airlines and Aviation Services']
['Food and Drink', 'Restaurants', 'Fast Food']
['Food and Drink', 'Restaurants', 'Coffee Shop']
['Food and Drink', 'Restaurants']
['Transfer', 'Credit']
['Travel', 'Airlines and Aviation Services']
['Travel', 'Taxi']
['Food and Drink', 'Restaurants']
['Payment']
['Food and Drink', 'Restaurants', 'Fast

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [192]:
# YOUR CODE HERE
account_summary_df = pd.DataFrame(columns = ['date', 'name', 'amount', 'category'])
transactions = transaction_response['transactions']
for x, transaction in enumerate(transactions):
    account_summary_df.loc[x] = [transaction['date'], transaction['name'], transaction['amount'], transaction['category'][0]]

account_summary_df.head()

,date,name,amount,category
0,2020-06-14,CREDIT CARD 3333 PAYMENT *//,25,Payment
1,2020-06-14,Uber,5.4,Travel
2,2020-06-13,ACH Electronic CreditGUSTO PAY 123456,5850,Transfer
3,2020-06-13,CD DEPOSIT .INITIAL.,1000,Transfer
4,2020-06-12,Touchstone Climbing,78.5,Recreation


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [193]:
# YOUR CODE HERE
account_summary_df.dtypes


date        object
name        object
amount      object
category    object
dtype: object

In [194]:
account_summary_df['date'] = pd.to_datetime(account_summary_df['date'], format = '%Y-%m-%d', errors = 'coerce')
account_summary_df['amount'] = account_summary_df['amount'].astype('float')

In [195]:
account_summary_df.dtypes

date        datetime64[ns]
name                object
amount             float64
category            object
dtype: object

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [196]:
# YOUR CODE HERE
income = client.Income.get(access_token)
pretty_print_response(income)

{
    "income": {
        "income_streams": [
            {
                "confidence": 0.99,
                "days": 720,
                "monthly_income": 500,
                "name": "UNITED AIRLINES"
            }
        ],
        "last_year_income": 6000,
        "last_year_income_before_tax": 7285,
        "max_number_of_overlapping_income_streams": 1,
        "number_of_income_streams": 1,
        "projected_yearly_income": 6085,
        "projected_yearly_income_before_tax": 7389
    },
    "request_id": "CQ8a8fZ3bfjXH9v"
}


In [197]:
print(income['income']['last_year_income'])

6000


In [198]:
print(income['income']['income_streams'][0]['monthly_income'])

500


In [199]:
print(income['income']['projected_yearly_income'])

6085


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [200]:
# YOUR CODE HERE
category_totals = account_summary_df.groupby('category').sum()
category_totals

,amount
category,
Food and Drink,3317.19
Payment,6310.50
Recreation,235.50
Shops,1500.00
Transfer,20537.34
Travel,35.19


### Calculate the expenses per month

In [201]:
account_summary_df['month'] = pd.DatetimeIndex(account_summary_df['date']).month
account_summary_df.head()

,date,name,amount,category,month
0,2020-06-14,CREDIT CARD 3333 PAYMENT *//,25.0,Payment,6
1,2020-06-14,Uber,5.4,Travel,6
2,2020-06-13,ACH Electronic CreditGUSTO PAY 123456,5850.0,Transfer,6
3,2020-06-13,CD DEPOSIT .INITIAL.,1000.0,Transfer,6
4,2020-06-12,Touchstone Climbing,78.5,Recreation,6


In [202]:
monthly_expenses = account_summary_df.groupby('month').sum()
monthly_expenses

,amount
month,
3,4084.83
4,10645.24
5,10645.24
6,6560.41
